In [1]:
import pandas as pd
import numpy as np
import os

### Extract CSVs into DataFrames

In [2]:
tweets_file = os.path.join("Resources", "trumptweets.csv")


In [3]:
approval_file = os.path.join("Resources", "approval.csv")


In [4]:
tweets_df = pd.read_csv(tweets_file)
tweets_df.head()

,Unnamed: 0,date,target,insult,tweet
0,1,2014-10-09,thomas-frieden,fool,"Can you believe this fool, Dr. Thomas Frieden ..."
1,2,2014-10-09,thomas-frieden,DOPE,"Can you believe this fool, Dr. Thomas Frieden ..."
2,3,2015-06-16,politicians,all talk and no action,Big time in U.S. today - MAKE AMERICA GREAT AG...
3,4,2015-06-24,ben-cardin,It's politicians like Cardin that have destroy...,Politician @SenatorCardin didn't like that I s...
4,5,2015-06-24,neil-young,total hypocrite,"For the nonbeliever, here is a photo of @Neily..."


In [5]:
approval_df = pd.read_csv(approval_file)
approval_df.head()

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Donald Trump,All polls,1/20/2021,1/20/2017,1/22/2017,Morning Consult,B/C,1992.0,rv,0.680029,...,37.0,45.686784,38.055805,NaN,NaN,http://static.politico.com/9b/13/82a3baf542ae9...,49249,77261,1/23/2017,11:47:59 20 Jan 2021
1,Donald Trump,All polls,1/20/2021,1/20/2017,1/22/2017,Gallup,B,1500.0,a,0.262323,...,45.0,45.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,11:47:59 20 Jan 2021
2,Donald Trump,All polls,1/20/2021,1/20/2017,1/24/2017,Ipsos,B-,1632.0,a,0.153481,...,45.2,43.451563,43.780389,NaN,T,http://polling.reuters.com/#poll/CP3_2/,49426,77599,3/1/2017,11:47:59 20 Jan 2021
3,Donald Trump,All polls,1/20/2021,1/21/2017,1/23/2017,Gallup,B,1500.0,a,0.242845,...,46.0,45.861441,44.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49262,77274,1/24/2017,11:47:59 20 Jan 2021
4,Donald Trump,All polls,1/20/2021,1/22/2017,1/24/2017,Gallup,B,1500.0,a,0.227380,...,45.0,46.861441,43.539189,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49236,77248,1/25/2017,11:47:59 20 Jan 2021


### Transform tweets DataFrame

In [6]:
tweetdates_df = tweets_df[["date", "target"]].copy()
tweetdates_df.head()

,date,target
0,2014-10-09,thomas-frieden
1,2014-10-09,thomas-frieden
2,2015-06-16,politicians
3,2015-06-24,ben-cardin
4,2015-06-24,neil-young


In [7]:
grouped_tweets = tweetdates_df.groupby(by="date").count()
grouped_tweets.head()

,target
date,
2014-10-09,2
2015-06-16,1
2015-06-24,3
2015-06-25,11
2015-06-26,3


### Transform approval DataFrame

In [8]:
approvalfinal_df = approval_df[["startdate","enddate","pollster","samplesize","adjusted_approve","adjusted_disapprove","url"]].copy()
approvalfinal_df.head()

,startdate,enddate,pollster,samplesize,adjusted_approve,adjusted_disapprove,url
0,1/20/2017,1/22/2017,Morning Consult,1992.0,45.686784,38.055805,http://static.politico.com/9b/13/82a3baf542ae9...
1,1/20/2017,1/22/2017,Gallup,1500.0,45.861441,43.539189,http://www.gallup.com/poll/201617/gallup-daily...
2,1/20/2017,1/24/2017,Ipsos,1632.0,43.451563,43.780389,http://polling.reuters.com/#poll/CP3_2/
3,1/21/2017,1/23/2017,Gallup,1500.0,45.861441,44.539189,http://www.gallup.com/poll/201617/gallup-daily...
4,1/22/2017,1/24/2017,Gallup,1500.0,46.861441,43.539189,http://www.gallup.com/poll/201617/gallup-daily...


In [9]:
approvalfinal_df["startdate"] = pd.to_datetime(approvalfinal_df.startdate)
approvalfinal_df["enddate"] = pd.to_datetime(approvalfinal_df.enddate)
approvalfinal_df.head()

,startdate,enddate,pollster,samplesize,adjusted_approve,adjusted_disapprove,url
0,2017-01-20,2017-01-22,Morning Consult,1992.0,45.686784,38.055805,http://static.politico.com/9b/13/82a3baf542ae9...
1,2017-01-20,2017-01-22,Gallup,1500.0,45.861441,43.539189,http://www.gallup.com/poll/201617/gallup-daily...
2,2017-01-20,2017-01-24,Ipsos,1632.0,43.451563,43.780389,http://polling.reuters.com/#poll/CP3_2/
3,2017-01-21,2017-01-23,Gallup,1500.0,45.861441,44.539189,http://www.gallup.com/poll/201617/gallup-daily...
4,2017-01-22,2017-01-24,Gallup,1500.0,46.861441,43.539189,http://www.gallup.com/poll/201617/gallup-daily...


### Create database connection

In [ ]:
rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

### Load DataFrames into database

In [ ]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

In [ ]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

###  Confirm data has been added

In [ ]:
pd.read_sql_query('select * from customer_name', con=engine).head()

In [ ]:
pd.read_sql_query('select * from customer_location', con=engine).head()